# **Bank Management System - Design & Implementation**

Here's a comprehensive **Bank Management System** design that covers real-world banking operations, security, and customer service needs.

---

## **1. System Overview**

A modern bank management system should handle:

- **Customer Accounts** (Savings, Current, Loans)
- **Transactions** (Deposits, Withdrawals, Transfers)
- **User Authentication & Security**
- **Reporting & Analytics**
- **Loan Processing**
- **ATM & Online Banking Integration**

---

## **2. Key Modules**

### **🟢 1. Customer Management**

- **Account Creation**
  - Personal/Business accounts
  - KYC (ID proof, address verification)
- **Account Types**
  - Savings, Current, Fixed Deposit, Recurring Deposit
- **Customer Dashboard**
  - View balance, transaction history, statements

### **🟢 2. Transaction Processing**

- **Deposits & Withdrawals**
  - Cash/Cheque/Online transfers
- **Fund Transfers**
  - NEFT, RTGS, IMPS, UPI
- **Transaction Limits**
  - Daily withdrawal limits, fraud detection

### **🟢 3. Loan Management**

- **Loan Types**
  - Personal, Home, Car, Education
- **Loan Approval Workflow**
  - Application → Verification → Approval/Rejection
- **EMI Calculator & Payment Tracking**

### **🟢 4. Security & Authentication**

- **Multi-Factor Authentication (MFA)**
  - OTP, Biometrics (Fingerprint/Face ID)
- **Fraud Detection**
  - AI-based anomaly detection
- **Role-Based Access Control (RBAC)**
  - Admin, Teller, Customer, Auditor

### **🟢 5. Reporting & Analytics**

- **Transaction Reports**
- **Customer Insights**
- **Regulatory Compliance** (RBI, GDPR)

### **🟢 6. ATM & Online Banking**

- **ATM Simulation**
  - Cash withdrawal, PIN change
- **Mobile/Web Banking**
  - Balance check, fund transfer, bill payments

---

## **3. Database Schema (Simplified)**

```sql
-- Customers Table
CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(100),
    phone VARCHAR(15),
    email VARCHAR(100),
    address TEXT,
    aadhar_number VARCHAR(20),
    pan_number VARCHAR(20),
    created_at TIMESTAMP
);

-- Accounts Table
CREATE TABLE Accounts (
    account_id INT PRIMARY KEY,
    customer_id INT,
    account_type ENUM('Savings', 'Current', 'FD', 'RD'),
    balance DECIMAL(15,2),
    interest_rate DECIMAL(5,2),
    status ENUM('Active', 'Inactive', 'Dormant'),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

-- Transactions Table
CREATE TABLE Transactions (
    transaction_id INT PRIMARY KEY,
    account_id INT,
    type ENUM('Deposit', 'Withdrawal', 'Transfer'),
    amount DECIMAL(15,2),
    transaction_date TIMESTAMP,
    description TEXT,
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
);

-- Loans Table
CREATE TABLE Loans (
    loan_id INT PRIMARY KEY,
    customer_id INT,
    loan_type ENUM('Personal', 'Home', 'Car', 'Education'),
    amount DECIMAL(15,2),
    interest_rate DECIMAL(5,2),
    tenure_months INT,
    status ENUM('Pending', 'Approved', 'Rejected'),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
```

---

## **4. Sample Python Implementation (OOP Approach)**

```python
import sqlite3
from datetime import datetime

class BankManagementSystem:
    def __init__(self, db_name="bank.db"):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self._create_tables()

    def _create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Customers (
                customer_id INTEGER PRIMARY KEY,
                name TEXT,
                phone TEXT,
                email TEXT,
                address TEXT,
                aadhar_number TEXT,
                pan_number TEXT,
                created_at TIMESTAMP
            )
        ''')
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Accounts (
                account_id INTEGER PRIMARY KEY,
                customer_id INTEGER,
                account_type TEXT,
                balance REAL,
                interest_rate REAL,
                status TEXT,
                FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
            )
        ''')
        self.conn.commit()

    def add_customer(self, name, phone, email, address, aadhar, pan):
        self.cursor.execute('''
            INSERT INTO Customers (name, phone, email, address, aadhar_number, pan_number, created_at)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (name, phone, email, address, aadhar, pan, datetime.now()))
        self.conn.commit()
        return self.cursor.lastrowid

    def create_account(self, customer_id, account_type, initial_deposit=0):
        interest_rate = 4.0 if account_type == "Savings" else 0.0
        self.cursor.execute('''
            INSERT INTO Accounts (customer_id, account_type, balance, interest_rate, status)
            VALUES (?, ?, ?, ?, ?)
        ''', (customer_id, account_type, initial_deposit, interest_rate, "Active"))
        self.conn.commit()
        return self.cursor.lastrowid

    def deposit(self, account_id, amount):
        self.cursor.execute('''
            UPDATE Accounts SET balance = balance + ? WHERE account_id = ?
        ''', (amount, account_id))
        self.conn.commit()

    def withdraw(self, account_id, amount):
        self.cursor.execute('''
            SELECT balance FROM Accounts WHERE account_id = ?
        ''', (account_id,))
        balance = self.cursor.fetchone()[0]
        if balance >= amount:
            self.cursor.execute('''
                UPDATE Accounts SET balance = balance - ? WHERE account_id = ?
            ''', (amount, account_id))
            self.conn.commit()
            return True
        return False  # Insufficient balance

    def transfer(self, from_account, to_account, amount):
        if self.withdraw(from_account, amount):
            self.deposit(to_account, amount)
            return True
        return False

# Example Usage
if __name__ == "__main__":
    bank = BankManagementSystem()
    customer_id = bank.add_customer(
        "John Doe", "9876543210", "john@example.com",
        "123 Main St", "123412341234", "ABCDE1234F"
    )
    account_id = bank.create_account(customer_id, "Savings", 1000)
    bank.deposit(account_id, 500)
    bank.withdraw(account_id, 200)
```

---

## **5. Security Considerations**

1. **Encryption**
   - AES-256 for sensitive data (account numbers, PAN)
2. **Rate Limiting**
   - Prevent brute-force attacks on login
3. **Audit Logs**
   - Track all transactions & admin actions
4. **Regular Backups**
   - Automated daily backups to secure cloud storage

---

## **6. Future Enhancements**

- **AI-Powered Chatbot** for customer queries
- **Blockchain Integration** for secure transactions
- **Predictive Analytics** for loan risk assessment
- **Voice-Based Banking** (Alexa/Google Assistant)

---

### **🚀 Deployment Options**

| Method                | Pros                        | Cons                      |
| --------------------- | --------------------------- | ------------------------- |
| **On-Premise**        | Full control, high security | High maintenance cost     |
| **Cloud (AWS/Azure)** | Scalable, low setup cost    | Monthly subscription fees |
| **Hybrid**            | Best of both worlds         | Complex configuration     |

---


Certainly! Below is a **Bank Management System** written in Python that allows users to create accounts, deposit and withdraw money, check balance, and manage multiple users. This version will include features such as **file I/O** for saving account details and **basic error handling**.

### **Bank Management System**

### 1. **Bank Account Class**

This class represents a single bank account with attributes for account holder name, balance, and account type (savings or checking).

```python
import json
import os

class BankAccount:
    def __init__(self, account_holder, balance=0.0, account_type="savings"):
        self.account_holder = account_holder
        self.balance = balance
        self.account_type = account_type

    def deposit(self, amount):
        """Deposit money into the account."""
        if amount <= 0:
            print("Deposit amount must be greater than zero.")
        else:
            self.balance += amount
            print(f"Deposited {amount}. Current balance: {self.balance}")

    def withdraw(self, amount):
        """Withdraw money from the account."""
        if amount <= 0:
            print("Withdrawal amount must be greater than zero.")
        elif amount > self.balance:
            print("Insufficient funds!")
        else:
            self.balance -= amount
            print(f"Withdrew {amount}. Current balance: {self.balance}")

    def get_balance(self):
        """Get the current balance of the account."""
        return self.balance

    def __str__(self):
        """Return account details as a string."""
        return f"Account Holder: {self.account_holder}, Type: {self.account_type}, Balance: {self.balance}"

    def to_dict(self):
        """Convert the bank account details to a dictionary for JSON storage."""
        return {
            "account_holder": self.account_holder,
            "balance": self.balance,
            "account_type": self.account_type
        }

    @staticmethod
    def from_dict(data):
        """Create a BankAccount instance from a dictionary."""
        return BankAccount(data["account_holder"], data["balance"], data["account_type"])
```

### 2. **Bank Class (Managing Multiple Accounts)**

This class manages multiple bank accounts and handles file operations to save and load data.

```python
class Bank:
    def __init__(self, filename="accounts.json"):
        self.filename = filename
        self.accounts = self.load_accounts()

    def load_accounts(self):
        """Load accounts from the JSON file."""
        accounts = {}
        if os.path.exists(self.filename):
            try:
                with open(self.filename, 'r') as file:
                    data = json.load(file)
                    for account_data in data:
                        account = BankAccount.from_dict(account_data)
                        accounts[account.account_holder] = account
            except json.JSONDecodeError:
                print("Error: Invalid file format or corrupted file.")
            except Exception as e:
                print(f"Error loading accounts: {e}")
        return accounts

    def save_accounts(self):
        """Save accounts to the JSON file."""
        try:
            with open(self.filename, 'w') as file:
                json.dump([account.to_dict() for account in self.accounts.values()], file, indent=4)
        except Exception as e:
            print(f"Error saving accounts: {e}")

    def create_account(self, account_holder, account_type="savings"):
        """Create a new account."""
        if account_holder in self.accounts:
            print("Account already exists for this holder.")
        else:
            new_account = BankAccount(account_holder, 0.0, account_type)
            self.accounts[account_holder] = new_account
            self.save_accounts()
            print(f"Account created for {account_holder}.")

    def deposit(self, account_holder, amount):
        """Deposit money into an account."""
        if account_holder in self.accounts:
            self.accounts[account_holder].deposit(amount)
            self.save_accounts()
        else:
            print("Account not found.")

    def withdraw(self, account_holder, amount):
        """Withdraw money from an account."""
        if account_holder in self.accounts:
            self.accounts[account_holder].withdraw(amount)
            self.save_accounts()
        else:
            print("Account not found.")

    def check_balance(self, account_holder):
        """Check the balance of an account."""
        if account_holder in self.accounts:
            print(f"Current balance for {account_holder}: {self.accounts[account_holder].get_balance()}")
        else:
            print("Account not found.")

    def list_accounts(self):
        """List all bank accounts."""
        if not self.accounts:
            print("No accounts available.")
        else:
            for account in self.accounts.values():
                print(account)

    def is_account_exists(self, account_holder):
        """Check if an account exists."""
        return account_holder in self.accounts
```

### 3. **Main Program (User Interaction)**

This part handles user input and interacts with the `Bank` and `BankAccount` classes.

```python
def print_menu():
    """Print the main menu."""
    print("\n--- Bank Management System ---")
    print("1. Create Account")
    print("2. Deposit Money")
    print("3. Withdraw Money")
    print("4. Check Balance")
    print("5. List All Accounts")
    print("6. Exit")
    print("-----------------------------")

def main():
    bank = Bank()

    while True:
        print_menu()
        choice = input("Enter your choice: ")

        if choice == '1':
            account_holder = input("Enter account holder name: ").strip()
            account_type = input("Enter account type (savings/checking): ").strip().lower()
            if account_type not in ["savings", "checking"]:
                print("Invalid account type. Defaulting to 'savings'.")
                account_type = "savings"
            bank.create_account(account_holder, account_type)

        elif choice == '2':
            account_holder = input("Enter account holder name: ").strip()
            if bank.is_account_exists(account_holder):
                try:
                    amount = float(input("Enter amount to deposit: ").strip())
                    bank.deposit(account_holder, amount)
                except ValueError:
                    print("Invalid amount. Please enter a valid number.")
            else:
                print("Account not found.")

        elif choice == '3':
            account_holder = input("Enter account holder name: ").strip()
            if bank.is_account_exists(account_holder):
                try:
                    amount = float(input("Enter amount to withdraw: ").strip())
                    bank.withdraw(account_holder, amount)
                except ValueError:
                    print("Invalid amount. Please enter a valid number.")
            else:
                print("Account not found.")

        elif choice == '4':
            account_holder = input("Enter account holder name: ").strip()
            if bank.is_account_exists(account_holder):
                bank.check_balance(account_holder)
            else:
                print("Account not found.")

        elif choice == '5':
            bank.list_accounts()

        elif choice == '6':
            print("Exiting the Bank Management System.")
            break

        else:
            print("Invalid choice, please try again.")

if __name__ == "__main__":
    main()
```

---

### **Features of the Bank Management System**

1. **File-based Storage**:

   - The bank accounts are saved in a JSON file (`accounts.json`) to ensure persistence.
   - Accounts are loaded into memory when the system starts and saved back to the file when modified.

2. **Creating Accounts**:

   - Users can create a bank account with a name and an account type (either savings or checking).
   - The account holder's name is used as the unique identifier for each account.

3. **Deposits and Withdrawals**:

   - Users can deposit and withdraw money from their accounts.
   - Input validation is in place to ensure that deposits and withdrawals are valid (positive amounts, sufficient funds).

4. **Checking Balance**:

   - Users can check the balance of any account by entering the account holder’s name.

5. **List All Accounts**:

   - All accounts are displayed with their account holder names, account types, and balances.

6. **Error Handling**:
   - The system handles errors like invalid deposits/withdrawals and invalid account names.
   - It also checks whether the account exists before performing operations on it.

---

### **Sample Output**:

#### Example 1: Create Account and Deposit

```
--- Bank Management System ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. List All Accounts
6. Exit
-----------------------------
Enter your choice: 1
Enter account holder name: John Doe
Enter account type (savings/checking): savings
Account created for John Doe.

--- Bank Management System ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. List All Accounts
6. Exit
-----------------------------
Enter your choice: 2
Enter account holder name: John Doe
Enter amount to deposit: 500
Deposited 500.0. Current balance: 500.0
```

#### Example 2: Withdraw and Check Balance

```
--- Bank Management System ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. List All Accounts
6. Exit
-----------------------------
Enter your choice: 3
Enter account holder name: John Doe
Enter amount to withdraw: 200
Withdrew 200.0. Current balance: 300.0

--- Bank Management System ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. List All Accounts
6. Exit
-----------------------------
Enter your choice: 4
Enter account holder name: John Doe
Current balance for John Doe: 300.0
```

#### Example 3: List Accounts

```
--- Bank Management System ---
1. Create Account
2. Deposit Money
3. Withdraw Money
4. Check Balance
5. List All Accounts
6. Exit
-----------------------------
Enter your choice: 5
Account Holder: John Doe, Type: savings, Balance: 300.0
```

---

### **Conclusion**:

This **Bank Management System** allows you to create accounts, deposit and withdraw money, check balances, and list accounts. It utilizes **JSON** to store account data persistently and incorporates basic error handling and input validation. You can extend this system further by adding features like transaction history, account transfer, and more advanced security features.
